In [97]:
# recommender function 
import pandas as pd
import numpy as np

In [98]:
movies_original = pd.read_csv('movies.csv')
ratings_original = pd.read_csv('ratings.csv')
links_original = pd.read_csv('links.csv')


In [102]:
# Create a new dataframe with movieId, title and genres
movies_df = movies_original.merge(ratings_original, on='movieId', how='inner')
# Predator (1987) with movieId == 3527
#sparse matrix
movies_crosstab_original = movies_df.pivot_table(
    index='userId', columns='movieId', values='rating')
movies_crosstab =movies_crosstab_original.copy()


In [103]:
def item_based_recommender(movie_name, n):
    
    movies = movies_original.copy()
    ratings = ratings_original.copy()
    movies['title_lower'] = movies['title'].str.lower()
    if len(movies.loc[movies['title_lower'].str.contains(movie_name.lower())]) ==0:
        return "Movie not found"
    else:
        top_popular_for_movieId = movies.loc[movies['title_lower'].str.contains( movie_name.lower())]['movieId'].values[0]
        

        # # Create a new dataframe with movieId, title and genres
        # movies_df = movies.merge(ratings, on='movieId', how='inner')
        # # Predator (1987) with movieId == 3527
        # #sparse matrix
        # movies_crosstab = movies_df.pivot_table(
        #     index='userId', columns='movieId', values='rating')
        # movies_crosstab = movies_crosstab_original.copy()
        predator_ratings = movies_crosstab[top_popular_for_movieId]
        predator_ratings[predator_ratings >= 0]  # exclude NaNs
        similar_to_predator = movies_crosstab.corrwith(predator_ratings)
        # NaN means that no users rated both the movie and the predator
        # drop the NaNs
        corr_predator = pd.DataFrame(similar_to_predator, columns=['Viewer'])
        corr_predator.dropna(inplace=True)
        corr_predator.sort_values('Viewer', ascending=False)
        rating = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
        rating['rating_count'] = ratings.groupby('movieId')['rating'].count()
        predator_corr_summary = corr_predator.join(rating['rating_count'])
        predator_corr_summary.drop(top_popular_for_movieId, inplace=True)
        top = predator_corr_summary[predator_corr_summary['rating_count'] >= 10].sort_values(
            'Viewer', ascending=False).head(n)

        
        return top.merge(movies, on='movieId', how='inner')[
            ['title', 'genres', 'Viewer']]



item_based_recommender('MaTRix', 4)



c:\Users\mrpal\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\mrpal\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,title,genres,Viewer
0,Nixon (1995),Drama,1.000000
1,Murder in the First (1995),Drama|Thriller,1.000000
2,Circle of Friends (1995),Drama|Romance,0.983739
3,"American in Paris, An (1951)",Musical|Romance,0.968330


In [95]:
item_based_recommender('harry Popper', 4)


'Movie not found'

In [96]:
item_based_recommender('harry potter', 9)


c:\Users\mrpal\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\mrpal\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,title,genres,Viewer
0,Pet Sematary (1989),Horror,1.0
1,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama,1.0
2,Coco (2017),Adventure|Animation|Children,1.0
3,Cold Comfort Farm (1995),Comedy,1.0
4,"Cook the Thief His Wife & Her Lover, The (1989)",Comedy|Drama,1.0
5,Secrets & Lies (1996),Drama,1.0
6,Pacific Heights (1990),Mystery|Thriller,1.0
7,"Madness of King George, The (1994)",Comedy|Drama,1.0
8,Kazaam (1996),Children|Comedy|Fantasy,1.0
